In [1]:
import pandas as pd # analysis
import numpy as np # calculation
from pandas import Series, DataFrame
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn import ensemble

In [2]:
# read data set
data = pd.read_csv('data.csv') # read training data

stocks = []
stock_codes = list(data.iloc[:, 0].unique())

for code in stock_codes:
    df = data[data['股票代码_Stkcd'] == code]
    stocks.append(df)


In [3]:
for i in range(len(stocks)):
    data = stocks[i]
    # delete the stock code
    data = data.iloc[:,1:]
    # split to x and y
    X = data.iloc[:, 1:]
    y = data.iloc[:, 0]
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.2)
    params = {'n_estimators': 500, 'max_depth': 6, 'min_samples_split': 2,
          'learning_rate': 0.0001, 'loss': 'ls'}
    clf = ensemble.GradientBoostingRegressor(**params)

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    print('-----------------------------------')
    print('Stock Code: ', stock_codes[i])
    
    mse = mean_squared_error(y_test, y_pred)
    print("MSE: %.6f" % mse)

    me = mean_absolute_error(y_test, y_pred)
    print('me: ', me)
    

-----------------------------------
Stock Code:  672
MSE: 0.000864
me:  0.022254413208880976
-----------------------------------
Stock Code:  63
MSE: 0.002332
me:  0.03510784910731284
-----------------------------------
Stock Code:  2302
MSE: 0.000830
me:  0.019880293914334563
-----------------------------------
Stock Code:  2307
MSE: 0.001288
me:  0.02550267491394151
-----------------------------------
Stock Code:  2773
MSE: 0.000897
me:  0.02380382597393028
-----------------------------------
Stock Code:  2792
MSE: 0.000489
me:  0.016340216038168293
-----------------------------------
Stock Code:  2821
MSE: 0.001110
me:  0.02607829585128513
-----------------------------------
Stock Code:  600487
MSE: 0.000935
me:  0.023239817111308344
-----------------------------------
Stock Code:  600535
MSE: 0.000809
me:  0.0221444664914731
-----------------------------------
Stock Code:  600545
MSE: 0.000132
me:  0.008671968058587031
-----------------------------------
Stock Code:  600585
MSE: 0.

In [ ]:
# data = data[['日收益率_Dret', '滞后一期收益', '市盈率_PE', '市净率_PB', '市销率_PS', '每股收益(摊薄)(元/股)_EPS', '每股营业利润(元/股)_OpPrfPS', '每股净资产(元/股)_NAPS', '每股营业收入_IncomePS']]

# test_data = test_data[['日收益率_Dret', '滞后一期收益', '市盈率_PE', '市净率_PB', '市销率_PS', '每股收益(摊薄)(元/股)_EPS', '每股营业利润(元/股)_OpPrfPS', '每股净资产(元/股)_NAPS', '每股营业收入_IncomePS']]
# data.head()

In [ ]:
# split into training set and test set
# data.rename("")
X_train = data.iloc[:, 1:]
y_train = data.iloc[:, 0]
X_test = test_data.iloc[:, 1:]
y_test = test_data.iloc[:, 0]

X_test.head()
# X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, test_size=0.33)

In [ ]:
params = {'n_estimators': 500, 'max_depth': 6, 'min_samples_split': 2,
          'learning_rate': 0.001, 'loss': 'ls'}
clf = ensemble.GradientBoostingRegressor(**params)

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
# mse = mean_squared_error(y_test, y_pred)
# print("MSE: %.6f" % mse)

# from sklearn.metrics import mean_absolute_error
# me = mean_absolute_error(y_test, y_pred)
# print('me: ', me)


In [ ]:
# d = {'y': y_test, 'y_pred': y_pred, 'diff': abs(y_test - y_pred)}
# df = pd.DataFrame(data=d)
# temp = df[df['diff'] < me]
# print('accuracy', (len(temp)/ len(y_test)) * 100)

In [ ]:
for i in y_pred:
    print(i)

In [ ]:
test_data['日收益率_Dret'] = y_pred
test_data.to_csv('prediction.csv')